# Part VII — Data, Integrations, and Advanced ORM  
## 32. Advanced Model Patterns (Abstract Bases, Proxy Models, Generic Relations, Soft Deletes, Audit Logging)

This chapter is about the model patterns that show up in **real** Django codebases
once the app grows beyond simple CRUD:

- “We need shared fields on many models” → **abstract base models**
- “We want a different behavior/view of the same table” → **proxy models**
- “We want to attach the same concept to multiple models” → **generic relations**
- “We can’t hard-delete records” → **soft deletes**
- “We need an audit trail: who changed what” → **audit logging**

You will learn:
- what each pattern is
- when to use it (and when *not* to)
- how it affects the DB schema and performance
- how to integrate it with admin, services, and tests

---

## 32.0 Learning Outcomes

By the end you should be able to:

1. Choose correctly between:
   - abstract base model
   - proxy model
   - multi-table inheritance
2. Build reusable “mixins” for common fields (timestamps, user tracking).
3. Implement a proxy model to create a specialized admin and queryset for a subset
   of rows without creating a new table.
4. Use ContentTypes / GenericForeignKey to attach a model (e.g., AuditLog or
   Reaction) to multiple target models.
5. Implement soft delete safely (managers, querysets, admin integration).
6. Understand the pitfalls:
   - referential integrity limitations of GenericFK
   - uniqueness constraints with soft deletes
   - bulk operations bypassing `save()`
7. Implement a production-grade audit log pattern and connect it to service-layer
   workflows.

---

## 32.1 Model Inheritance Options in Django (The Big Map)

Django supports three main “inheritance” patterns:

### 32.1.1 Abstract base classes (most common, safest)
- No separate DB table for the base class.
- Fields are copied into each child table.
- Great for shared fields and small shared methods.

Use for:
- `created_at` / `updated_at`
- `created_by` / `updated_by`
- `deleted_at`
- “tenant/org foreign key” patterns (sometimes)

### 32.1.2 Proxy models (same DB table, different Python behavior)
- No new table.
- Used to change:
  - default ordering
  - default manager/queryset
  - admin presentation
  - methods
- Great for “PublishedArticle” vs “DraftArticle” views of the same `Article` table.

### 32.1.3 Multi-table inheritance (creates a new table per subclass)
- Base model has its own table.
- Each subclass has its own table with a OneToOne link to base table.
- Queries often require joins.
- Harder to evolve, can surprise you in performance and migration complexity.

Use only when:
- you truly need “is-a” relationships with different schemas
- and you’re okay with join overhead and complexity

**Industry guidance:**  
Use abstract bases and proxies frequently. Use multi-table inheritance rarely.

---

## 32.2 Abstract Base Models (Reusable Fields Without Extra Tables)

### 32.2.1 The classic example: timestamps
You already use timestamps in multiple models (`Article`, `Task`, `Comment`,
`TaskEvent`, `TaskExportJob`, etc.). Centralizing them prevents drift.

Create `core/models.py` (create a `core` app if you don’t have one; recommended for
shared base classes):

```bash
python manage.py startapp core
```

Add to `INSTALLED_APPS`:

```python
INSTALLED_APPS += ["core"]
```

Now create `core/models.py`:

```python
from __future__ import annotations

from django.db import models
from django.utils import timezone


class TimeStampedModel(models.Model):
    created_at = models.DateTimeField(default=timezone.now, editable=False)
    updated_at = models.DateTimeField(auto_now=True)

    class Meta:
        abstract = True
```

#### Why `abstract = True` matters
It tells Django:
- do not create a table for `TimeStampedModel`
- just copy its fields into subclasses

### 32.2.2 Apply it to a model (example: Tag)
Update `articles/models.py`:

```python
from core.models import TimeStampedModel


class Tag(TimeStampedModel):
    name = models.CharField(max_length=50, unique=True)
    slug = models.SlugField(max_length=60, unique=True)

    class Meta:
        ordering = ["name"]

    def __str__(self) -> str:
        return self.name
```

Make migrations:

```bash
python manage.py makemigrations
python manage.py migrate
```

#### What you just changed at DB level
- `articles_tag` table now has `created_at`, `updated_at`
- no new `core_timestampedmodel` table exists

### 32.2.3 User-tracking base model (created_by / updated_by)
For multi-user business apps (tasks), it’s common to require audit fields.

Create `core/models_usertrack.py` (or keep in `core/models.py`):

```python
from __future__ import annotations

from django.conf import settings
from django.db import models


class UserTrackedModel(models.Model):
    created_by = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.PROTECT,
        related_name="%(app_label)s_%(class)s_created",
    )
    updated_by = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.PROTECT,
        related_name="%(app_label)s_%(class)s_updated",
    )

    class Meta:
        abstract = True
```

#### Explain `related_name="%(app_label)s_%(class)s_created"`
This is a Django pattern for abstract bases:
- it prevents related_name collisions across multiple subclasses
- Django replaces placeholders with app/model names

### 32.2.4 When *not* to use abstract bases
Avoid abstract bases when:
- the shared fields are rarely used and complicate schema
- you need a real polymorphic query across all subclasses (then you might consider
  multi-table inheritance, but still be cautious)

---

## 32.3 Proxy Models (Same Table, Different Behavior)

Proxy models are excellent for:
- editorial workflows (“Published only”)
- admin separation (“Archived tasks”)
- internal tools that want different ordering/default filters

### 32.3.1 Example: PublishedArticle proxy
In `articles/models.py` add:

```python
class PublishedArticle(Article):
    class Meta:
        proxy = True
        ordering = ["-published_at", "-created_at"]

    @classmethod
    def get_queryset(cls):
        # This method is not used by Django automatically; prefer a custom manager.
        return Article.objects.published()
```

This defines a proxy but doesn’t yet change query behavior. The better approach is
a custom manager on the proxy:

```python
from django.db import models


class PublishedArticleManager(models.Manager):
    def get_queryset(self):
        return super().get_queryset().filter(status=Article.Status.PUBLISHED)


class PublishedArticle(Article):
    objects = PublishedArticleManager()

    class Meta:
        proxy = True
        ordering = ["-published_at", "-created_at"]
```

#### What you get
- `PublishedArticle.objects.all()` returns only published rows.
- It still uses the same DB table as `Article`.
- No migrations are created (no schema change).

### 32.3.2 Proxy models in admin (very common)
Register a separate admin for `PublishedArticle` to give editors a focused view:

```python
# articles/admin.py
from django.contrib import admin

from .models import Article, PublishedArticle


@admin.register(PublishedArticle)
class PublishedArticleAdmin(admin.ModelAdmin):
    list_display = ("id", "title", "slug", "published_at", "author")
    ordering = ("-published_at",)
```

Now admin shows:
- “Articles” (all)
- “Published articles” (filtered)  
Both point to the same table.

### 32.3.3 When *not* to use proxy models
Avoid proxies if:
- you need a different schema/table (then it’s not a proxy problem)
- you can solve it with a normal queryset and don’t need separate admin behavior

---

## 32.4 Multi-table Inheritance (Pitfalls and Real Use Cases)

Multi-table inheritance creates multiple tables and joins automatically.

### 32.4.1 Example (conceptual): Payment methods
```python
from django.db import models


class PaymentMethod(models.Model):
    created_at = models.DateTimeField(auto_now_add=True)


class CardPaymentMethod(PaymentMethod):
    last4 = models.CharField(max_length=4)
    brand = models.CharField(max_length=20)


class BankPaymentMethod(PaymentMethod):
    bank_name = models.CharField(max_length=100)
```

DB tables:
- `paymentmethod`
- `cardpaymentmethod` (OneToOne to paymentmethod)
- `bankpaymentmethod` (OneToOne to paymentmethod)

### 32.4.2 Why it’s often painful
- common queries require joins
- deleting involves multiple tables
- migrations and constraints are more complex
- “polymorphic queries” are not as clean as people assume

**Industry preference:**  
Use multi-table inheritance only when it models a true domain “is-a” hierarchy and
you accept the costs. Many teams instead use:
- a single table with a “type” field and nullable fields, or
- separate models with explicit OneToOne relationships, or
- polymorphic libraries (extra dependency) only when necessary

---

## 32.5 Generic Relations (ContentTypes): One Model That Can Point to Many Models

Generic relations are powered by `django.contrib.contenttypes`.

### 32.5.1 When generic relations are appropriate
Use generic relations for cross-cutting concepts like:
- audit logs
- comments/notes attached to multiple models
- reactions/likes
- attachments
- notifications referencing different object types

### 32.5.2 When generic relations are NOT appropriate
Avoid them when:
- you need strict DB-level referential integrity (foreign keys)
- you need high-performance joins across the relationship frequently
- you need complex constraints across the target model

GenericForeignKey is not a real FK in the DB, so the DB cannot enforce “target
exists.”

---

## 32.6 Implement a Real Generic Audit Log (Production-Useful Pattern)

You already have `TaskEvent` which is task-specific. That’s great.
But many teams also add a generic AuditLog for:
- “who did what” across many models
- security and compliance
- debugging and incident forensics

### 32.6.1 Enable contenttypes app (already enabled by default)
Make sure this is in `INSTALLED_APPS` (it is in default Django projects):

```python
"django.contrib.contenttypes",
```

### 32.6.2 Create `audit` app
```bash
python manage.py startapp audit
```

Add to `INSTALLED_APPS`:

```python
INSTALLED_APPS += ["audit"]
```

### 32.6.3 Create `audit/models.py`
```python
from __future__ import annotations

from django.conf import settings
from django.contrib.contenttypes.fields import GenericForeignKey
from django.contrib.contenttypes.models import ContentType
from django.db import models
from django.utils import timezone


class AuditLog(models.Model):
    """
    Generic audit log event.

    IMPORTANT LIMITATION:
    content_type + object_id is not a real FK, so referential integrity is not
    enforced by the DB. Your code must avoid dangling references.
    """

    created_at = models.DateTimeField(default=timezone.now, editable=False)

    actor = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.SET_NULL,
        null=True,
        blank=True,
        related_name="audit_logs",
    )

    action = models.CharField(max_length=50)

    # Generic target reference:
    content_type = models.ForeignKey(ContentType, on_delete=models.PROTECT)
    object_id = models.CharField(max_length=64)
    target = GenericForeignKey("content_type", "object_id")

    # Optional structured details:
    details = models.JSONField(default=dict, blank=True)

    # Useful request correlation:
    request_id = models.CharField(max_length=64, blank=True)

    class Meta:
        ordering = ["-created_at"]
        indexes = [
            models.Index(fields=["content_type", "object_id", "-created_at"]),
            models.Index(fields=["actor", "-created_at"]),
            models.Index(fields=["action", "-created_at"]),
        ]

    def __str__(self) -> str:
        return f"{self.action} {self.content_type_id}:{self.object_id}"
```

Make migrations:

```bash
python manage.py makemigrations
python manage.py migrate
```

### 32.6.4 Create a helper to log events (service function)
Create `audit/services.py`:

```python
from __future__ import annotations

from django.contrib.contenttypes.models import ContentType

from audit.models import AuditLog
from config.request_context import get_request_id


def log_event(*, actor, action: str, target, details: dict | None = None) -> AuditLog:
    ct = ContentType.objects.get_for_model(target.__class__)
    return AuditLog.objects.create(
        actor=actor if getattr(actor, "is_authenticated", False) else None,
        action=action,
        content_type=ct,
        object_id=str(target.pk),
        details=details or {},
        request_id=get_request_id(),
    )
```

#### Why include request_id
You already built request IDs and structured logging.
Audit logs become far more useful when you can correlate:
- request logs
- audit events
- error tracking

---

## 32.7 Hook Audit Logging into Your Existing Workflows (Service Layer)

### 32.7.1 Example: log Task status changes in `tasks/services.py`
In your `update_task_from_form` service (after detecting change), add:

```python
from audit.services import log_event
```

Then:

```python
if old_status != task.status:
    log_event(
        actor=actor,
        action="task.status_changed",
        target=task,
        details={"from": old_status, "to": task.status},
    )
```

Do similar for:
- assignee changes
- task creation
- export start / export done

**Important:** Don’t log raw secrets or entire request bodies in `details`.

---

## 32.8 Soft Deletes (Don’t Actually Delete Rows)

Soft delete means:
- you keep the row in DB
- you mark it as deleted (usually `deleted_at` timestamp)
- “normal queries” hide deleted rows

### 32.8.1 Why teams use soft deletes
- compliance: keep records
- safety: accidental deletion recovery
- audit: maintain history
- “undo delete” feature
- referential integrity: other records reference it

### 32.8.2 The big cost of soft deletes
Soft delete complicates:
- uniqueness constraints
- queries (must filter out deleted)
- indexes (need partial indexes for “alive” rows)
- foreign key semantics (a “deleted” row still exists)
- cleanup policies (do you ever hard-delete?)

**Industry guidance:**  
Soft delete only where the business truly needs it. Don’t soft delete everything by
default.

---

## 32.9 Implement Soft Delete Correctly (QuerySet + Manager + Model)

We’ll implement soft delete for **Comment** (great example: you may want to hide a
comment without losing it).

### 32.9.1 Create reusable soft delete base classes in `core/models_softdelete.py`
```python
from __future__ import annotations

from django.db import models
from django.utils import timezone


class SoftDeleteQuerySet(models.QuerySet):
    def alive(self):
        return self.filter(deleted_at__isnull=True)

    def dead(self):
        return self.filter(deleted_at__isnull=False)

    def delete(self):
        # Soft-delete bulk queryset deletes:
        return super().update(deleted_at=timezone.now())

    def hard_delete(self):
        # True delete:
        return super().delete()


class SoftDeleteManager(models.Manager):
    def get_queryset(self):
        return SoftDeleteQuerySet(self.model, using=self._db).alive()

    def hard_delete(self):
        return self.get_queryset().hard_delete()


class SoftDeleteModel(models.Model):
    deleted_at = models.DateTimeField(null=True, blank=True)

    objects = SoftDeleteManager()
    all_objects = SoftDeleteQuerySet.as_manager()

    class Meta:
        abstract = True

    def delete(self, using=None, keep_parents=False):
        self.deleted_at = timezone.now()
        self.save(update_fields=["deleted_at"])

    def hard_delete(self, using=None, keep_parents=False):
        return super().delete(using=using, keep_parents=keep_parents)

    @property
    def is_deleted(self) -> bool:
        return self.deleted_at is not None
```

#### Explanation of design choices
- `objects` hides deleted rows by default (this is what most code will use).
- `all_objects` lets admins/maintenance code see everything.
- Overriding `QuerySet.delete()` is critical:
  - Without it, `Comment.objects.filter(...).delete()` would hard-delete rows.
- Providing `hard_delete()` keeps an escape hatch for cleanup jobs.

### 32.9.2 Apply soft delete to Comment
Update `articles/models.py`:

```python
from core.models_softdelete import SoftDeleteModel


class Comment(SoftDeleteModel):
    article = models.ForeignKey(
        Article,
        on_delete=models.CASCADE,
        related_name="comments",
    )
    name = models.CharField(max_length=80)
    body = models.TextField(...)
    is_approved = models.BooleanField(default=False)
    created_at = models.DateTimeField(auto_now_add=True)

    class Meta:
        ordering = ["created_at"]
        indexes = [
            models.Index(fields=["article", "is_approved", "created_at"]),
            models.Index(fields=["deleted_at"]),
        ]
```

Run migrations:

```bash
python manage.py makemigrations
python manage.py migrate
```

### 32.9.3 Update comment queries to exclude deleted (often automatic)
Because `objects` manager hides deleted rows, this query:

```python
article.comments.filter(is_approved=True)
```

uses the default manager on Comment via the related manager and should respect it.
But **be careful**: related managers can use base manager depending on configuration.
To be explicit, you can filter `deleted_at__isnull=True` in high-risk places.

Example (in views/services):

```python
comments_qs = article.comments.filter(
    is_approved=True,
    deleted_at__isnull=True,
).order_by("created_at")
```

This is explicit and safe even if you later change manager behavior.

---

## 32.10 Soft Delete and Uniqueness Constraints (Important Pitfall)

If you soft delete records, you may want to allow “re-creating” a row with the same
unique value later.

Example:
- Article slug is unique.
- If you soft delete an article, do you want to reuse slug?
  - Sometimes yes (restore, re-publish).
  - Sometimes no (URLs must never be reused).

If you want slug reuse with soft delete, you need a **conditional unique
constraint** (Postgres partial unique index):

```python
models.UniqueConstraint(
    fields=["slug"],
    name="unique_slug_alive",
    condition=Q(deleted_at__isnull=True),
)
```

But this works only on DBs that support conditional unique constraints properly
(PostgreSQL). SQLite behavior differs.

**Industry recommendation:**
- For public URLs (slugs), often **do not** allow reuse; keep slug unique forever.
- For internal identifiers, conditional uniqueness is more common.

---

## 32.11 Admin Integration for Soft Deletes (So Staff Can Recover)

### 32.11.1 Show deleted rows in admin (optional)
In `articles/admin.py`, for CommentAdmin, override queryset:

```python
from django.contrib import admin
from articles.models import Comment


@admin.register(Comment)
class CommentAdmin(admin.ModelAdmin):
    list_display = ("id", "article", "name", "is_approved", "deleted_at", "created_at")
    list_filter = ("is_approved", "deleted_at", "created_at")
    search_fields = ("name", "body", "article__slug")

    def get_queryset(self, request):
        qs = super().get_queryset(request)
        # super() uses default manager; ensure we see all:
        return Comment.all_objects.select_related("article")
```

### 32.11.2 Add admin actions: soft delete / restore / hard delete
```python
from django.utils import timezone


@admin.action(description="Soft delete selected comments")
def soft_delete_comments(modeladmin, request, queryset):
    queryset.update(deleted_at=timezone.now())


@admin.action(description="Restore selected comments")
def restore_comments(modeladmin, request, queryset):
    queryset.update(deleted_at=None)


@admin.action(description="Hard delete selected comments (dangerous)")
def hard_delete_comments(modeladmin, request, queryset):
    # Use all_objects if needed; queryset here already from all_objects
    queryset.delete()  # careful: if this queryset is SoftDeleteQuerySet, delete() is soft
    # For true hard delete:
    Comment.all_objects.filter(id__in=queryset.values_list("id", flat=True)).hard_delete()
```

A cleaner hard-delete action is:

```python
@admin.action(description="Hard delete selected comments (dangerous)")
def hard_delete_comments(modeladmin, request, queryset):
    Comment.all_objects.filter(
        id__in=queryset.values_list("id", flat=True)
    ).hard_delete()
```

Attach actions:

```python
actions = [soft_delete_comments, restore_comments, hard_delete_comments]
```

#### Operational note
Hard delete should usually be restricted to superusers or removed entirely unless
you have a strict retention policy.

---

## 32.12 Generic Relations Example #2 (Optional): Reactions (Like) on Articles and Comments

This reinforces generic relations with a “user reaction” model.

### 32.12.1 Create `reactions` app
```bash
python manage.py startapp reactions
```

Add to `INSTALLED_APPS`:

```python
INSTALLED_APPS += ["reactions"]
```

### 32.12.2 `reactions/models.py`
```python
from __future__ import annotations

from django.conf import settings
from django.contrib.contenttypes.fields import GenericForeignKey
from django.contrib.contenttypes.models import ContentType
from django.db import models
from django.utils import timezone


class Reaction(models.Model):
    class Kind(models.TextChoices):
        LIKE = "like", "Like"

    created_at = models.DateTimeField(default=timezone.now, editable=False)

    user = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.CASCADE,
        related_name="reactions",
    )

    kind = models.CharField(max_length=20, choices=Kind.choices)

    content_type = models.ForeignKey(ContentType, on_delete=models.CASCADE)
    object_id = models.CharField(max_length=64)
    target = GenericForeignKey("content_type", "object_id")

    class Meta:
        constraints = [
            models.UniqueConstraint(
                fields=["user", "kind", "content_type", "object_id"],
                name="unique_reaction_per_target",
            )
        ]
        indexes = [
            models.Index(fields=["content_type", "object_id", "-created_at"]),
            models.Index(fields=["user", "-created_at"]),
        ]
```

Migrate:

```bash
python manage.py makemigrations
python manage.py migrate
```

### 32.12.3 Add GenericRelation for reverse access (nice but optional)
In `articles/models.py`:

```python
from django.contrib.contenttypes.fields import GenericRelation

class Article(...):
    reactions = GenericRelation("reactions.Reaction")
```

In `Comment`:

```python
reactions = GenericRelation("reactions.Reaction")
```

Now you can do:
- `article.reactions.count()`
- `comment.reactions.filter(kind="like")`

**Performance note:** This can still create N+1 if used in loops. Use annotation or
prefetch patterns if needed.

---

## 32.13 Testing These Patterns (High Value Tests)

### 32.13.1 Test soft delete hides comments
```python
from django.test import TestCase
from django.utils import timezone

from articles.models import Article, Comment
from tests.factories import ArticleFactory, UserFactory


class SoftDeleteTests(TestCase):
    def test_soft_deleted_comment_is_hidden(self):
        user = UserFactory()
        article = ArticleFactory(author=user)

        c = Comment.all_objects.create(
            article=article,
            name="A",
            body="Hello",
            is_approved=True,
        )

        self.assertEqual(Comment.objects.count(), 1)

        c.delete()  # soft delete
        self.assertEqual(Comment.objects.count(), 0)
        self.assertEqual(Comment.all_objects.count(), 1)
        self.assertTrue(Comment.all_objects.first().is_deleted)
```

### 32.13.2 Test audit log writes
```python
from django.test import TestCase

from audit.models import AuditLog
from audit.services import log_event
from tests.factories import TaskFactory, UserFactory


class AuditLogTests(TestCase):
    def test_log_event_creates_row(self):
        user = UserFactory()
        task = TaskFactory()

        log = log_event(
            actor=user,
            action="task.created",
            target=task,
            details={"title": task.title},
        )

        self.assertEqual(AuditLog.objects.count(), 1)
        self.assertEqual(log.action, "task.created")
```

---

# 32.14 Mini‑Lab (Recommended): Add Soft Delete + Audit Logging to a Real Workflow

## Goal
- Soft delete comments instead of hard delete.
- Create audit logs when:
  - comment submitted
  - comment approved
  - comment deleted/restored (admin action or view)

### Steps
1. Implement `SoftDeleteModel` and apply to Comment.
2. Update comment moderation admin actions:
   - approve
   - soft delete
   - restore
3. Add audit logging calls:
   - in comment submit view/service
   - in admin actions (log actor, action, target)
4. Add tests proving:
   - deleted comments don’t show publicly
   - restore brings them back
   - audit logs are created

---

## 32.15 Common Pitfalls and How to Avoid Them

### Pitfall A: GenericForeignKey allows dangling references
Because DB can’t enforce it, you can end up with audit logs pointing to objects that
no longer exist.

Mitigations:
- prefer soft delete for important targets
- periodic cleanup job for dangling references (optional)
- include enough details in AuditLog to be useful even if target is gone

### Pitfall B: Soft delete breaks uniqueness
If you want to reuse unique fields, you need conditional unique constraints (Postgres).
If you do not, you must accept “unique forever” semantics.

### Pitfall C: Bulk updates bypass `save()` and signals
Your service layer should handle audit logging explicitly. Don’t assume `save()` is
called in bulk operations (`queryset.update()`).

### Pitfall D: Soft delete + cascades
A DB-level `on_delete=CASCADE` does not “soft cascade.”
If you soft delete a parent, children remain unless your code handles it.

Strategy:
- soft delete should be applied consistently across related models if you need
  cascading semantics
- or explicitly soft delete children in services

---

## 32.16 Exercises (Do These Before Proceeding)

1. Implement a `PublishedArticle` proxy and register it in admin with a custom
   list display and filters.
2. Add a generic `Reaction` model and allow liking Articles and Comments.
   - Ensure uniqueness: one like per user per target.
3. Add an `AuditLog` entry whenever:
   - an export job is started
   - an export job finishes (DONE/FAILED)
4. Add a “hard delete cleanup” management command:
   - permanently delete soft-deleted comments older than 90 days
   - log how many were removed

---

## 32.17 Chapter Summary

- **Abstract base models** are the standard way to share fields and small behavior
  without creating extra tables.
- **Proxy models** give you multiple “views” of the same table (great for admin and
  domain-specific query behavior).
- **Multi-table inheritance** creates extra tables and joins; use sparingly.
- **Generic relations** are powerful for cross-cutting models (audit logs,
  reactions) but lack DB-enforced referential integrity.
- **Soft delete** requires correct managers/querysets and affects uniqueness and
  cascading semantics.
- A service-layer + audit log pattern is how professional Django apps maintain
  consistency across UI, APIs, background jobs, and admin.

---

Next chapter: **33. Search, Files, and External Services**  
We’ll integrate external APIs with timeouts/retries, build secure webhook receivers
(signature verification + replay protection), and implement file storage patterns
(S3-like backends) that scale beyond local disk.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='31. advanced_migrations_and_data_management.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='33. search_files_and_external_services.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
